In [1]:
import tensorflow as tf 
from tensorflow import keras
import tkinter as tk
import pickle
import re
import textwrap
from datetime import datetime
import numpy as np

## Loading in our model and tokenizer:

In [2]:
model=keras.models.load_model('BidenBot.h5')

#Pickling has some security issues so we will retrain our tokenizer on the same dataset:
def get_tokenizer():
    global tokenizer
    global max_id
    global dataset_size

    string_Biden='' #We will store the Biden dataset here

    with open('text_Biden.txt','r') as file:
        list_Biden=file.readlines()

    for line in list_Biden:
        string_Biden+=line

    tokenizer = keras.preprocessing.text.Tokenizer(char_level=True)
    tokenizer.fit_on_texts(string_Biden)

    max_id = len(tokenizer.word_index) # number of distinct characters
    dataset_size = tokenizer.document_count # total number of characters

get_tokenizer()

## Data Preprocessing:
Similar to what we did for training our RNN model, we will need to use tokenizer to create a list of sequences represented by a sparse one-hot tensor object.

In [5]:
def preprocess(texts):
    X = np.array(tokenizer.texts_to_sequences(texts)) - 1
    return tf.one_hot(X, max_id)

## Creating the output:
We will call the complete_text func. with a text variable which acts to initialize our string. 
For the number of characters requested (n_chars), we will create said number of characters by predicting the next char, n_chars times. To create unique strings, we will use a log function on our predicted probabilities before applying a tf.random.categorical function which uses such values in order to generate a set of values (in this case we only generate one sample). I will show you an example of how temperature affects our output:

In [6]:
#Generate a random value 40 times
temperature=.1
probability=.9
tf.random.categorical([[np.log(probability)/temperature, np.log(1-probability)/temperature]], num_samples=40).numpy()

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

In [7]:
temperature=5
probability=.9
tf.random.categorical([[np.log(probability)/temperature, np.log(1-probability)/temperature]], num_samples=40).numpy()

array([[1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0,
        0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0]])

So as you can see, we are able to directly make our model generate more random output just by increasing the temperature values. Let's apply this:

In [8]:
def complete_text(text, n_chars=50, temperature=1):
    for i in range(n_chars):
        text += next_char(text, temperature)
    return text
def next_char(text, temperature=1):
    X_new = preprocess([text])
    y_proba = model(X_new)[0, -1:, :]
    rescaled_logits = tf.math.log(y_proba) / temperature
    char_id = tf.random.categorical(rescaled_logits, num_samples=1) + 1
    return tokenizer.sequences_to_texts(char_id.numpy())[0]

We will now create functions necessary to run our tkinter code. This will call complete_text(), add new lines with add_n_lines(), so as to not have the output run off the tk window, and capitalize every letter that comes after the end of a sentence. click_generate() will activate when we press the button on our tk window.

In [9]:
def click_generate(entry):
    label2['text']=output(entry)

def replace_lower_case(speech):
    #We will replace all lower case characters after period_space, period_question mark, period_exclamation.
    indeces=[i.end() for i in re.finditer('(\. *)|(\? *)|(\! *)', speech)] #Get the indices of the char proceeding a period,...
    for i in indeces:
        #Separate into three pieces and capitalize the letter. If not a letter then continue,
        try:
            string_pre=speech[:(i)]
            char_Upper=speech[i].capitalize() 
            string_post=speech[(i+1):]
            speech=string_pre+char_Upper+string_post
        except Exception as x:
            continue
    return speech

def add_n_lines(speech):
    #Add new lines:
    string=textwrap.dedent(speech)
    speech='\n'.join(i for line in string.splitlines() for i in textwrap.wrap(line, width=47))
    return speech

def output(entry):
    if len(entry)==0:
        return 'Must include text' #If no input given

    #You can play around with the temperature yourself. Anything between .01 and 1.0 will give you a relatively good output, with a higher value being more 'random', and with a lower value           allowing the model to more likely choose the letter arrangement that it believes to be correct (which often leads to repetition unfortunately)!
    speech=complete_text(entry,n_chars=460, temperature=0.17)
    speech=replace_lower_case(speech)
    speech=add_n_lines(str(speech))

    speech+= '\n \n \t \t\t - BidenBot' #Add a signature
    print(speech)
    return speech

## Running this may take several minutes, depending on what hardware you are using!

Let's now create our window. I have included images for the background, trying to keep it from being political despite the political nature of the training data. Sometimes, if the image doesn't load, you have to close the window and run the code again.

In [12]:
root=tk.Tk()

canvas=tk.Canvas(root, height=750, width=540)
canvas.pack()

try:
    background_image=tk.PhotoImage(file='Biden.png')
    background_label=tk.Label(root, image=background_image)
    background_label.place(relwidth=1, relheight=1)

except Exception as x:
    print(x)

frame=tk.Frame(root, bg='#1a1aff', bd=4)
frame.place(relx=0.5, rely=0.02, relwidth=.9, relheight=0.05, anchor='n')

label1=tk.Label(frame, text='Start output with:', bg='#e6ffff', font=30)
label1.place(relx=0, rely=0, relwidth=0.3, relheight=1)

entry=tk.Entry(frame, bg='#e6ffff', font=50)
entry.place(relx=0.3, rely=0, relwidth=.5, relheight=1)

button=tk.Button(frame, text='Generate', bg='blue', fg='blue', font=78, command=lambda: click_generate(entry.get()))
button.place(relx=.8, rely=0, relwidth=.2, relheight=1)

bottom_frame=tk.Frame(root, bg='#1a1aff', bd=4)
bottom_frame.place(relx=0.5, rely=.2, relwidth=.8, relheight=.4, anchor='n')

label2=tk.Label(bottom_frame, bg='#e6ffff', font=('Microsoft Sans Serif', 20), anchor='nw', justify='left')
label2.place(relwidth=1, relheight=1)

root.mainloop()

hello it the state of americans have all seen
the pastpone. I’m going to shop and the reason
when you say, i’m running as a proud democrat,
but i tell you what, the past seek with you.
That sense of empowerment with you. Thank you.
Thank you.
 thank you and thank you for the soul of the
palllies families and surrendered to the moon
in the state of americans have all seed the
state of americans have all see nothing. But i
tell you what, the past say, “joey, and we
 
 	 		 - BidenBot
